<a href="https://www.kaggle.com/code/sairamya18/ensemble-cnn?scriptVersionId=166179213" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import os
import zipfile
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam,RMSprop,SGD,Adagrad,Nadam

In [ ]:
import os
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Dataset path
dataset_path = "/kaggle/input/fer2013-cleaned-dataset/images1"

# Constants
num_classes = 7
input_shape = (48, 48, 1)
batch_size = 64
epochs = 100
ensemble_size = 5

# Data Augmentation
train_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator = ImageDataGenerator(rescale=1./255)

# Load train and test data
train_data = train_data_generator.flow_from_directory(
    os.path.join(dataset_path, 'train'),
    color_mode='grayscale',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_data = test_data_generator.flow_from_directory(
    os.path.join(dataset_path, 'test'),
    color_mode='grayscale',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Define ensemble models
ensemble_models = []

# Train multiple CNN models
for i in range(ensemble_size):
    # Define and compile the CNN model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Train the model with early stopping
    history = model.fit(
        train_data,
        steps_per_epoch=len(train_data),
        epochs=epochs,
        validation_data=test_data,
        validation_steps=len(test_data),
        callbacks=[early_stopping],  # Pass the early stopping callback
        verbose=1
    )
    
    # Add the trained model to the ensemble
    ensemble_models.append(model)

# Predict with each model
individual_predictions = []
for model in ensemble_models:
    predictions = model.predict(test_data)
    individual_predictions.append(predictions)

# Combine predictions (e.g., average)
ensemble_predictions = np.mean(individual_predictions, axis=0)

# Evaluate ensemble performance
ensemble_loss, ensemble_accuracy = model.evaluate(test_data)
print("Ensemble Loss:", ensemble_loss)
print("Ensemble Accuracy:", ensemble_accuracy)


Found 28044 images belonging to 7 classes.
Found 7177 images belonging to 7 classes.
Epoch 1/100


2024-03-09 04:24:47.763377: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout_4/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


439/439 [==============================] - 51s 105ms/step - loss: 1.9101 - accuracy: 0.3385 - val_loss: 1.8811 - val_accuracy: 0.3513
Epoch 2/100
439/439 [==============================] - 46s 104ms/step - loss: 1.4096 - accuracy: 0.4638 - val_loss: 1.2888 - val_accuracy: 0.5102
Epoch 3/100
439/439 [==============================] - 46s 105ms/step - loss: 1.2730 - accuracy: 0.5202 - val_loss: 1.2427 - val_accuracy: 0.5277
Epoch 4/100
439/439 [==============================] - 46s 105ms/step - loss: 1.1977 - accuracy: 0.5472 - val_loss: 1.2154 - val_accuracy: 0.5380
Epoch 5/100
439/439 [==============================] - 47s 107ms/step - loss: 1.1383 - accuracy: 0.5725 - val_loss: 1.1685 - val_accuracy: 0.5630
Epoch 6/100
439/439 [==============================] - 47s 108ms/step - loss: 1.0908 - accuracy: 0.5911 - val_loss: 1.2049 - val_accuracy: 0.5610
Epoch 7/100
439/439 [==============================] - 50s 115ms/step - loss: 1.0604 - accuracy: 0.6022 - val_loss: 1.1433 - val_accurac

2024-03-09 04:42:37.374977: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/dropout_8/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


439/439 [==============================] - 51s 105ms/step - loss: 1.9065 - accuracy: 0.3321 - val_loss: 1.7492 - val_accuracy: 0.3563
Epoch 2/100
439/439 [==============================] - 46s 105ms/step - loss: 1.4414 - accuracy: 0.4509 - val_loss: 1.3261 - val_accuracy: 0.4955
Epoch 3/100
439/439 [==============================] - 46s 105ms/step - loss: 1.2936 - accuracy: 0.5100 - val_loss: 1.3416 - val_accuracy: 0.4804
Epoch 4/100
439/439 [==============================] - 47s 106ms/step - loss: 1.2139 - accuracy: 0.5405 - val_loss: 1.6790 - val_accuracy: 0.3984
Epoch 5/100
439/439 [==============================] - 46s 105ms/step - loss: 1.1575 - accuracy: 0.5634 - val_loss: 1.1379 - val_accuracy: 0.5717
Epoch 6/100
439/439 [==============================] - 46s 105ms/step - loss: 1.1174 - accuracy: 0.5787 - val_loss: 1.1421 - val_accuracy: 0.5738
Epoch 7/100
439/439 [==============================] - 46s 106ms/step - loss: 1.0693 - accuracy: 0.5961 - val_loss: 2.0333 - val_accurac

...

439/439 [==============================] - 52s 108ms/step - loss: 1.9154 - accuracy: 0.3320 - val_loss: 1.7149 - val_accuracy: 0.3582
Epoch 2/100
439/439 [==============================] - 48s 109ms/step - loss: 1.4451 - accuracy: 0.4516 - val_loss: 1.4542 - val_accuracy: 0.4537
Epoch 3/100
439/439 [==============================] - 47s 107ms/step - loss: 1.2932 - accuracy: 0.5077 - val_loss: 1.3274 - val_accuracy: 0.4971
Epoch 4/100
439/439 [==============================] - 47s 107ms/step - loss: 1.2135 - accuracy: 0.5420 - val_loss: 1.3032 - val_accuracy: 0.5182
Epoch 5/100
439/439 [==============================] - 46s 105ms/step - loss: 1.1486 - accuracy: 0.5678 - val_loss: 1.2047 - val_accuracy: 0.5493
Epoch 6/100
439/439 [==============================] - 47s 106ms/step - loss: 1.1043 - accuracy: 0.5838 - val_loss: 1.1534 - val_accuracy: 0.5890
Epoch 7/100
439/439 [==============================] - 47s 107ms/step - loss: 1.0645 - accuracy: 0.6008 - val_loss: 1.1713 - val_accurac

2024-03-09 05:35:17.011702: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_5/dropout_20/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


439/439 [==============================] - 53s 109ms/step - loss: 1.8934 - accuracy: 0.3259 - val_loss: 1.6140 - val_accuracy: 0.3828
Epoch 2/100
439/439 [==============================] - 47s 107ms/step - loss: 1.4344 - accuracy: 0.4502 - val_loss: 1.3149 - val_accuracy: 0.4891
Epoch 3/100
359/439 [=======================>......] - ETA: 6s - loss: 1.2840 - accuracy: 0.5158

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



439/439 [==============================] - 46s 105ms/step - loss: 0.8278 - accuracy: 0.6932 - val_loss: 1.1567 - val_accuracy: 0.5937
Epoch 14/100
439/439 [==============================] - 50s 113ms/step - loss: 0.8078 - accuracy: 0.6983 - val_loss: 1.3206 - val_accuracy: 0.6131
Epoch 15/100
439/439 [==============================] - 47s 106ms/step - loss: 0.7491 - accuracy: 0.7212 - val_loss: 1.1480 - val_accuracy: 0.6085
Epoch 16/100
439/439 [==============================] - 46s 105ms/step - loss: 0.7149 - accuracy: 0.7371 - val_loss: 1.2295 - val_accuracy: 0.6139
Epoch 17/100
439/439 [==============================] - 46s 105ms/step - loss: 0.6712 - accuracy: 0.7500 - val_loss: 1.2066 - val_accuracy: 0.6152
Epoch 18/100
439/439 [==============================] - 47s 107ms/step - loss: 0.6429 - accuracy: 0.7624 - val_loss: 1.1818 - val_accuracy: 0.6047
Epoch 19/100
439/439 [==============================] - 47s 108ms/step - loss: 0.6063 - accuracy: 0.7745 - val_loss: 1.5967 - val_a

In [ ]:

dataset_path = "/kaggle/input/fer2013-cleaned-dataset/images1"

num_classes = 7

input_shape = (48, 48, 1)

batch_size = 64


In [ ]:
from keras.models import Sequential
import numpy as np
import tensorflow as tf
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization


model = Sequential()


model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


In [ ]:
train_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator = ImageDataGenerator(rescale=1./255)

train_data = train_data_generator.flow_from_directory(
    os.path.join(dataset_path, 'train'),
    color_mode='grayscale',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_data = test_data_generator.flow_from_directory(
    os.path.join(dataset_path, 'test'),
    color_mode='grayscale',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
num_train_samples = 28044
num_test_samples = 7177

In [ ]:
# history = model.fit(
#    train_data,
#      steps_per_epoch=num_train_samples // batch_size,
#      epochs=2
# )

In [ ]:
# history = model.fit(
#    train_data,
#      steps_per_epoch=num_train_samples // batch_size,
#      epochs=5
# )

In [ ]:
# history = model.fit(
#    train_data,
#      steps_per_epoch=num_train_samples // batch_size,
#      epochs=5
# )

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)


In [ ]:
history = model.fit(
   train_data,
     steps_per_epoch=num_train_samples // batch_size,
     epochs=100,
    callbacks=[early_stopping],/6
    batch_size=batch_size
)

In [ ]:
test_loss, test_accuracy = model.evaluate(
    test_data,
    steps=num_test_samples // batch_size
)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train'], loc='upper left')

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train'], loc='upper left')

plt.show()


In [ ]:
# Get the true labels
true_labels = test_data.classes

# Get the predicted labels
predicted_labels = model.predict(test_data).argmax(axis=1)

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
plt.imshow(cm, cmap='Blues')

# Add colorbar and labels
plt.colorbar()
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

plt.show()
